In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import random
from dataset import extract_matrix_users_movies_ratings, extract_users_movies_ratings_lists
from libreco.data import split_by_ratio_chrono, DatasetPure
from libreco.algorithms import ALS
from sklearn.metrics import mean_squared_error
from scipy.sparse import coo_matrix

import importlib
import dataset
importlib.reload(dataset)
from dataset import extract_matrix_users_movies_ratings, extract_users_movies_ratings_lists

Instructions for updating:
non-resource variables are not supported in the long term


In [70]:
data_pd = pd.read_csv('../data/data_train.csv')
data_matrix, _ = extract_matrix_users_movies_ratings(data_pd)
# axis 0 means: we subtract the means for each movie

data_matrix[data_matrix == 0] = 'nan'
MEAN = np.nanmean(data_matrix, keepdims=True)
data_matrix_normalized = np.subtract(data_matrix, MEAN, where=data_matrix != 0)
MEAN_MOVIE_MATRIX = np.nanmean(data_matrix_normalized, axis=0, keepdims=True)
data_matrix_normalized = np.subtract(data_matrix_normalized, MEAN_MOVIE_MATRIX, where=data_matrix != 0)
MEAN_USER_MATRIX = np.nanmean(data_matrix_normalized, axis=1, keepdims=True)
data_matrix_normalized = np.subtract(data_matrix_normalized, MEAN_USER_MATRIX, where=data_matrix != 0)
#STD = np.nanstd(data_matrix, keepdims=True)

STD_MATRIX = np.nanstd(data_matrix, axis=0, keepdims=True)

In [58]:
def normalize_extract_lists(pd, users, movies):  
    # axis 0 means: we subtract the means for each movie
    a = np.zeros(len(users))
    #data_matrix_normalized = np.divide(data_matrix_normalized, STD_MATRIX, where=data_matrix != 0)

    for i, (user, movie) in enumerate(zip(users, movies)):
        a[i] = data_matrix_normalized[user][movie]
    return a

In [26]:
def normalize_lists(pd, users, movies):
    data_matrix, _ = extract_matrix_users_movies_ratings(pd)
    # axis 0 means: we subtract the means for each movie
    a = np.zeros(len(users))
    data_matrix_normalized = np.subtract(data_matrix, MEAN, where=data_matrix != 0)
    #data_matrix_normalized = np.divide(data_matrix_normalized, STD, where=data_matrix != 0)
        
    for i, (user, movie) in enumerate(zip(users, movies)):
        a[i] = data_matrix_normalized[user][movie]
    return a

In [74]:
RANDOM_STATE = 58
train_pd, val_pd = train_test_split(data_pd, train_size=0.9, random_state=RANDOM_STATE)

users_train, movies_train, _ = extract_users_movies_ratings_lists(train_pd)
ratings_train = normalize_extract_lists(train_pd, users_train, movies_train)

users_val, movies_val, _ = extract_users_movies_ratings_lists(val_pd)
ratings_val = normalize_extract_lists(val_pd, users_val, movies_val)

train = pd.DataFrame({'user': users_train, 'item': movies_train, 'label': ratings_train})
val = pd.DataFrame({'user': users_val, 'item': movies_val, 'label': ratings_val})

# %%
train_data, data_info = DatasetPure.build_trainset(train)
eval_data = DatasetPure.build_evalset(val)

In [75]:
def get_mean_movie(i):
    mean_vector = np.squeeze(MEAN_MOVIE_MATRIX)
    return mean_vector[i]

def get_mean_user(i):
    mean_vector = np.squeeze(MEAN_USER_MATRIX)
    return mean_vector[i]


def get_std(i):
    std_vector = np.squeeze(STD_MATRIX)
    return std_vector[i]

def get_mean_movie_offset(movies):
    return np.array(list(map(get_mean_movie, movies)))

def get_mean_user_offset(users):
    return np.array(list(map(get_mean_user, users)))


def get_std_offset(movies):
    return np.array(list(map(get_std, movies)))

def transform_back(ratings, users, movies):
    return ratings + MEAN + get_mean_movie_offset(movies) + get_mean_user_offset(users)

In [16]:
def save_predictions(res_path, predictions):
    test_pd = pd.read_csv('../data/sampleSubmission.csv')
    test_pd = test_pd.astype({'Prediction': 'float'})
    test_pd.iloc[:, 1] = predictions
    test_pd.to_csv(res_path, index=False, float_format='%.3f')

In [76]:
emb_sizes = [2,4,8,16,32]
regs = [0.1, 0.5, 1, 5, 10]
alphas = [0.01, 0.1, 1, 10, 100]
david = np.zeros((125, len(users_val)))

for i, emb_size in enumerate(emb_sizes):
    for j, reg in enumerate(regs):
        for k, alpha in enumerate(alphas):
            giorgio = ALS(task="rating", data_info=data_info, embed_size=emb_size, n_epochs=5,
                        reg=reg, alpha=alpha, seed=42)
            giorgio.fit(train_data, verbose=2, use_cg=False, n_threads=8,
                    eval_data=eval_data, metrics=["rmse", "mae", "r2"])
            #david[len(regs)*len(alphas)*i+len(alphas)*j+k] = np.array(giorgio.predict(user=users_val, item=movies_val)) * get_std_offset(movies_val) + get_mean_offset(movies_val)
            david[len(regs)*len(alphas)*i+len(alphas)*j+k] = transform_back(np.array(giorgio.predict(user=users_val, item=movies_val)), users_val, movies_val)

            
#mean_squared_error(david.mean(axis=0), ratings_val * get_std_offset(movies_val) + get_mean_offset(movies_val), squared=False)
mean_squared_error(david.mean(axis=0), transform_back(ratings_val, users_val, movies_val)[0], squared=False)

Training start time: 2022-07-23 16:01:03
Epoch 1 elapsed: 0.073s


eval_pred: 100%|███████████████████████████████| 15/15 [00:00<00:00, 601.50it/s]

	 eval rmse: 0.9981
	 eval mae: 0.8030
	 eval r2: -0.0198


Epoch 2 elapsed: 0.065s


eval_pred: 100%|███████████████████████████████| 15/15 [00:00<00:00, 665.59it/s]


...

	 eval rmse: 1.0643
	 eval mae: 0.8347
	 eval r2: -0.1594


Epoch 5 elapsed: 0.342s


eval_pred: 100%|███████████████████████████████| 15/15 [00:00<00:00, 483.36it/s]

	 eval rmse: 1.0649
	 eval mae: 0.8340
	 eval r2: -0.1608


0.9821442983857993

In [66]:
transform_back(ratings_val, users_val, movies_val)[0]

array([5.26500517, 5.47754103, 9.04810396, ..., 7.63300936, 8.15324429,
       9.07768795])

In [53]:
mean_squared_error(david.mean(axis=0), transform_back(ratings_val, users_val, movies_val)[0], squared=False)

0.9886550705809852

In [77]:
#------------- submission ----------------#

emb_sizes = [2,4,8,16,32]
regs = [0.1, 0.5, 1, 5, 10]
alphas = [0.01, 0.1, 1, 10, 100]

train_pd = pd.read_csv(f'../data/data_train.csv')

users_train, movies_train, _ = extract_users_movies_ratings_lists(train_pd)
ratings_train = normalize_extract_lists(train_pd, users_train, movies_train)

test_pd = pd.read_csv('../data/sampleSubmission.csv')
users_test, movies_test, _ = extract_users_movies_ratings_lists(test_pd)

train = pd.DataFrame({'user': users_train, 'item': movies_train, 'label': ratings_train})

david = np.zeros((125, len(users_test)))

# %%
train_data, data_info = DatasetPure.build_trainset(train)

for i, emb_size in enumerate(emb_sizes):
    for j, reg in enumerate(regs):
        for k, alpha in enumerate(alphas):
            giorgio = ALS(task="rating", data_info=data_info, embed_size=emb_size, n_epochs=5,
                        reg=reg, alpha=alpha, seed=42)
            giorgio.fit(train_data, verbose=2, use_cg=False, n_threads=8, metrics=["rmse", "mae", "r2"])
            david[len(regs)*len(alphas)*i+len(alphas)*j+k] = transform_back(np.array(giorgio.predict(user=users_test, item=movies_test)), users_test, movies_test)
            
yhat = david.mean(axis=0)
save_predictions(f'./ALS-predictedSubmission.csv', yhat)

Training start time: 2022-07-23 16:05:27
Epoch 1 elapsed: 0.035s
Epoch 2 elapsed: 0.098s
Epoch 3 elapsed: 0.078s
Epoch 4 elapsed: 0.078s
Epoch 5 elapsed: 0.038s
Training start time: 2022-07-23 16:05:30
Epoch 1 elapsed: 0.051s
Epoch 2 elapsed: 0.054s
Epoch 3 elapsed: 0.059s
Epoch 4 elapsed: 0.059s
Epoch 5 elapsed: 0.045s
Training start time: 2022-07-23 16:05:34
Epoch 1 elapsed: 0.049s
Epoch 2 elapsed: 0.049s
Epoch 3 elapsed: 0.060s
Epoch 4 elapsed: 0.050s
Epoch 5 elapsed: 0.046s
Training start time: 2022-07-23 16:05:38
Epoch 1 elapsed: 0.046s
Epoch 2 elapsed: 0.074s
Epoch 3 elapsed: 0.087s
Epoch 4 elapsed: 0.042s
Epoch 5 elapsed: 0.057s
Training start time: 2022-07-23 16:05:42
Epoch 1 elapsed: 0.061s
Epoch 2 elapsed: 0.063s
Epoch 3 elapsed: 0.046s
Epoch 4 elapsed: 0.047s
Epoch 5 elapsed: 0.044s
Training start time: 2022-07-23 16:05:45
Epoch 1 elapsed: 0.051s
Epoch 2 elapsed: 0.079s
Epoch 3 elapsed: 0.048s
Epoch 4 elapsed: 0.066s
Epoch 5 elapsed: 0.080s
Training start time: 2022-07-23 16

Epoch 3 elapsed: 0.084s
Epoch 4 elapsed: 0.065s
Epoch 5 elapsed: 0.087s
Training start time: 2022-07-23 16:07:05
Epoch 1 elapsed: 0.074s
Epoch 2 elapsed: 0.063s
Epoch 3 elapsed: 0.082s
Epoch 4 elapsed: 0.082s
Epoch 5 elapsed: 0.066s
Training start time: 2022-07-23 16:07:08
Epoch 1 elapsed: 0.074s
Epoch 2 elapsed: 0.059s
Epoch 3 elapsed: 0.106s
Epoch 4 elapsed: 0.080s
Epoch 5 elapsed: 0.091s
Training start time: 2022-07-23 16:07:12
Epoch 1 elapsed: 0.081s
Epoch 2 elapsed: 0.078s
Epoch 3 elapsed: 0.084s
Epoch 4 elapsed: 0.065s
Epoch 5 elapsed: 0.073s
Training start time: 2022-07-23 16:07:16
Epoch 1 elapsed: 0.072s
Epoch 2 elapsed: 0.072s
Epoch 3 elapsed: 0.104s
Epoch 4 elapsed: 0.072s
Epoch 5 elapsed: 0.080s
Training start time: 2022-07-23 16:07:20
Epoch 1 elapsed: 0.088s
Epoch 2 elapsed: 0.067s
Epoch 3 elapsed: 0.079s
Epoch 4 elapsed: 0.072s
Epoch 5 elapsed: 0.082s
Training start time: 2022-07-23 16:07:24
Epoch 1 elapsed: 0.063s
Epoch 2 elapsed: 0.089s
Epoch 3 elapsed: 0.072s
Epoch 4 el

Training start time: 2022-07-23 16:08:42
Epoch 1 elapsed: 0.126s
Epoch 2 elapsed: 0.144s
Epoch 3 elapsed: 0.145s
Epoch 4 elapsed: 0.144s
Epoch 5 elapsed: 0.150s
Training start time: 2022-07-23 16:08:47
Epoch 1 elapsed: 0.122s
Epoch 2 elapsed: 0.129s
Epoch 3 elapsed: 0.145s
Epoch 4 elapsed: 0.130s
Epoch 5 elapsed: 0.128s
Training start time: 2022-07-23 16:08:51
Epoch 1 elapsed: 0.158s
Epoch 2 elapsed: 0.139s
Epoch 3 elapsed: 0.148s
Epoch 4 elapsed: 0.159s
Epoch 5 elapsed: 0.118s
Training start time: 2022-07-23 16:08:55
Epoch 1 elapsed: 0.129s
Epoch 2 elapsed: 0.147s
Epoch 3 elapsed: 0.138s
Epoch 4 elapsed: 0.121s
Epoch 5 elapsed: 0.167s
Training start time: 2022-07-23 16:08:59
Epoch 1 elapsed: 0.171s
Epoch 2 elapsed: 0.181s
Epoch 3 elapsed: 0.173s
Epoch 4 elapsed: 0.185s
Epoch 5 elapsed: 0.168s
Training start time: 2022-07-23 16:09:04
Epoch 1 elapsed: 0.162s
Epoch 2 elapsed: 0.124s
Epoch 3 elapsed: 0.133s
Epoch 4 elapsed: 0.121s
Epoch 5 elapsed: 0.157s
Training start time: 2022-07-23 16

Epoch 2 elapsed: 0.291s
Epoch 3 elapsed: 0.332s
Epoch 4 elapsed: 0.300s
Epoch 5 elapsed: 0.323s
Training start time: 2022-07-23 16:10:35
Epoch 1 elapsed: 0.267s
Epoch 2 elapsed: 0.291s
Epoch 3 elapsed: 0.289s
Epoch 4 elapsed: 0.290s
Epoch 5 elapsed: 0.272s
Training start time: 2022-07-23 16:10:40
Epoch 1 elapsed: 0.308s
Epoch 2 elapsed: 0.288s
Epoch 3 elapsed: 0.300s
Epoch 4 elapsed: 0.343s
Epoch 5 elapsed: 0.322s
Training start time: 2022-07-23 16:10:45
Epoch 1 elapsed: 0.343s
Epoch 2 elapsed: 0.292s
Epoch 3 elapsed: 0.318s
Epoch 4 elapsed: 0.346s
Epoch 5 elapsed: 0.293s
Training start time: 2022-07-23 16:10:50
Epoch 1 elapsed: 0.317s
Epoch 2 elapsed: 0.327s
Epoch 3 elapsed: 0.377s
Epoch 4 elapsed: 0.330s
Epoch 5 elapsed: 0.350s
Training start time: 2022-07-23 16:10:55
Epoch 1 elapsed: 0.292s
Epoch 2 elapsed: 0.325s
Epoch 3 elapsed: 0.303s
Epoch 4 elapsed: 0.298s
Epoch 5 elapsed: 0.302s
Training start time: 2022-07-23 16:11:00
Epoch 1 elapsed: 0.349s
Epoch 2 elapsed: 0.360s
Epoch 3 el

Epoch 4 elapsed: 0.325s
Epoch 5 elapsed: 0.341s
Training start time: 2022-07-23 16:12:43
Epoch 1 elapsed: 0.340s
Epoch 2 elapsed: 0.339s
Epoch 3 elapsed: 0.357s
Epoch 4 elapsed: 0.359s
Epoch 5 elapsed: 0.363s
Training start time: 2022-07-23 16:12:48
Epoch 1 elapsed: 0.350s
Epoch 2 elapsed: 0.358s
Epoch 3 elapsed: 0.371s
Epoch 4 elapsed: 0.375s
Epoch 5 elapsed: 0.360s
Training start time: 2022-07-23 16:12:54
Epoch 1 elapsed: 0.346s
Epoch 2 elapsed: 0.360s
Epoch 3 elapsed: 0.373s
Epoch 4 elapsed: 0.347s
Epoch 5 elapsed: 0.358s
Training start time: 2022-07-23 16:12:59
Epoch 1 elapsed: 0.412s
Epoch 2 elapsed: 0.459s
Epoch 3 elapsed: 0.427s
Epoch 4 elapsed: 0.438s
Epoch 5 elapsed: 0.379s
Training start time: 2022-07-23 16:13:05
Epoch 1 elapsed: 0.360s
Epoch 2 elapsed: 0.366s
Epoch 3 elapsed: 0.354s
Epoch 4 elapsed: 0.325s
Epoch 5 elapsed: 0.315s
Training start time: 2022-07-23 16:13:10
Epoch 1 elapsed: 0.324s
Epoch 2 elapsed: 0.343s
Epoch 3 elapsed: 0.336s
Epoch 4 elapsed: 0.359s
Epoch 5 el

In [78]:
yhat

array([3.21190344, 3.34355777, 3.77724277, ..., 3.05701952, 3.07623667,
       3.74655416])

In [79]:
yhat.std()

0.5818578183345156